In [82]:
# ADHD Classification Based on HRV
# Purpose: Determine whether heart rate variability can predict diagnosis of ADHD
# Author: Alexander Maksiaev

In [83]:
# Housekeeping

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
# from tsfresh import extract_features
# from tsfresh import extract_relevant_features
# from tsfresh import select_features
import featuretools as ft 
from featuretools.selection import (
    remove_highly_correlated_features,
    remove_highly_null_features,
    remove_single_value_features,
)
import glob
import os
import csv 
from datetime import datetime

# Directories

adhd_dir_raw = "C:/Users\maksi\Documents\Statistics\Projects\Movement_Mental_Health\hyperaktiv\hyperaktiv_with_controls\hyperaktiv_with_controls"
adhd_dir = adhd_dir_raw.replace("\\", "/")
print(adhd_dir)
# activity_dir = os.path.join(adhd_dir, "activity_data/")
hrv_dir = os.path.join(adhd_dir, "hrv_data")
print(hrv_dir)
output_dir = os.path.join(adhd_dir, "pt_features") 
# controls_dir = os.path.join(adhd_dir, "hyperaktiv_with_controls/hyperaktiv_with_controls/")
# activity_dir_controls = os.path.join(controls_dir, "activity_data/")

os.chdir(adhd_dir)

# Main files
patient_info = pd.read_csv("patient_info.csv", delimiter=";")
# features = pd.read_csv("features.csv", delimiter=";")
patient_info
# print(features)

# os.chdir(activity_dir)
# activity_data_01 = pd.read_csv("patient_activity_01.csv", delimiter=";")
# activity_data_01["ID"] = 1
# activity_data_01

# os.chdir(hrv_dir)
# hrv_data = pd.read_csv("patient_hr_1.csv", delimiter=";")
# hrv_data

C:/Users/maksi/Documents/Statistics/Projects/Movement_Mental_Health/hyperaktiv/hyperaktiv_with_controls/hyperaktiv_with_controls
C:/Users/maksi/Documents/Statistics/Projects/Movement_Mental_Health/hyperaktiv/hyperaktiv_with_controls/hyperaktiv_with_controls\hrv_data


,ID,SEX,AGE,ACC,ACC_TIME,ACC_DAYS,HRV,HRV_TIME,HRV_HOURS,CPT_II,...,HADS_D,MED,MED_Antidepr,MED_Moodstab,MED_Antipsych,MED_Anxiety_Benzo,MED_Sleep,MED_Analgesics_Opioids,MED_Stimulants,filter_$
0,1,0,3,1,16:00:00,6,1,11:00:00,21.0,0,...,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,2,0,4,1,10:54:00,6.8,0,NaN,NaN,1,...,7.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,3,1,2,1,15:28:00,7.2,1,15:25:00,21.0,1,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,4,1,3,0,NaN,NaN,1,16:55:00,22.0,1,...,6.0,1.0,1.0,NaN,NaN,1.0,NaN,1.0,NaN,0.0
4,5,1,1,1,14:24:00,5.9,1,16:00:00,12.0,1,...,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,236,1,2,1,9:30:00,"13,3",0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,237,0,1,1,15:00:00,14,0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,238,0,4,1,9:00:00,"14,6",0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,239,0,4,1,9:00:00,"14,3",0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# # Data initialization

# os.chdir(adhd_dir)

# controls_info = pd.read_csv("patient_info.csv", delimiter=";") # Includes patients as well
# # print(controls_info)

# adhd_patients = controls_info[controls_info["ADHD"] == 1.0] # ADHD only
# # print(adhd_patients)
# # print(controls_info.columns)

# controls = controls_info[np.sum(controls_info.loc[:, "ADHD":"OTHER"], axis=1) == 0] # Neurotypicals only
# non_adhd_controls = controls_info[controls_info["ADHD"] == 0] # Non-ADHD only
# # print(controls)
# # print(non_adhd_controls)


In [ ]:
# # Extract HRV features for each participant

# os.chdir(hrv_dir)

# def read_hrv_file(filepath, patient_id):
#     data = [ ]
#     with open(filepath) as f:
#         csv_reader = csv.reader(f, delimiter=";")
#         next(csv_reader)
#         for line in csv_reader:
#             data.append([ datetime.strptime(line[0], "%Y-%m-%d %H:%M:%S.%f").timestamp(), float(line[1].split(" ")[0])])
#     data = pd.DataFrame(data, columns=["TIME", "HRV"])
#     data["ID"] = patient_id
#     return data

# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# for filepath in glob.glob(os.path.join(hrv_dir, "*.csv")):
#     print("Reading %s" % filepath)
#     patient_id = os.path.splitext(os.path.basename(filepath))[0]
#     feature_filepath = os.path.join(output_dir, "%s_features.csv" % patient_id)

#     if os.path.exists(feature_filepath):
#         print("Skipped...")
#         continue
    
#     data = read_hrv_file(filepath, patient_id)

#     es = ft.EntitySet(id="pt_data")
#     es = es.add_dataframe(
#     dataframe_name="pt_data",
#     dataframe=data,
#     )
#     rolling_mean_primitive = RollingMean(
#         window_length=window_length, gap=gap, min_periods=window_length
#     )

#     rolling_min_primitive = RollingMin(
#         window_length=window_length, gap=gap, min_periods=window_length
#     )
#     feature_matrix, feature_defs = ft.dfs(
#     entityset=es,
#     target_dataframe_name="pt_data",
#     trans_primitives=(
#         datetime_primitives
#         + delaying_primitives
#         + [rolling_mean_primitive, rolling_min_primitive]
#     ),
#     )
#     print(feature_defs)
#     print(feature_matrix)
#     break 
#     # features.to_csv(feature_filepath, index=False, sep=";")    

Reading C:/Users/maksi/Documents/Statistics/Projects/Movement_Mental_Health/hyperaktiv/hyperaktiv_with_controls/hyperaktiv_with_controls\hrv_data\patient_hr_1.csv


c:\Users\maksi\AppData\Local\Programs\Python\Python311\Lib\site-packages\featuretools\entityset\entityset.py:1717: UserWarning: Using first column as index. To change this, specify the index parameter
  warnings.warn(
c:\Users\maksi\AppData\Local\Programs\Python\Python311\Lib\site-packages\featuretools\synthesis\deep_feature_synthesis.py:169: UserWarning: Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn(
c:\Users\maksi\AppData\Local\Programs\Python\Python311\Lib\site-packages\featuretools\synthesis\dfs.py:321: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['max', 'mean', 'min', 'skew', 'std']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data. If the DFS call contained multiple instances of a primitive in the list above, none of them were used.


[<Feature: HRV>, <Feature: ID>]
                  HRV            ID
TIME                               
1.236182e+09  2294.92  patient_hr_1
1.236182e+09   631.84  patient_hr_1
1.236182e+09   624.02  patient_hr_1
1.236182e+09   636.72  patient_hr_1
1.236182e+09   625.98  patient_hr_1
...               ...           ...
1.236258e+09   671.88  patient_hr_1
1.236258e+09   669.92  patient_hr_1
1.236258e+09   670.90  patient_hr_1
1.236258e+09   662.11  patient_hr_1
1.236258e+09   672.85  patient_hr_1

[107474 rows x 2 columns]


In [ ]:
# Combine feature csvs into one file for analysis 

In [ ]:
# # Concat all activity dataframes

# os.chdir(hrv_dir)

# csv_files = glob.glob("*.csv")

# # Create an empty dataframe to store the combined data
# # mega_activity_df = pd.DataFrame()

# # Loop through each CSV file and append it to the combined dataframe
# def get_activity_file(file):
# # for file in csv_files:
#     file_name_pieces = str.split(file, sep="_")
#     pt_id = file_name_pieces[-1][:-4] # Last part of the file name is patient id, without the .csv
#     df = pd.read_csv(file, delimiter=";")
#     df["ID"] = int(pt_id)
#     # mega_activity_df = pd.concat([mega_activity_df, df], ignore_index=True)
#     return df

# # mega_activity_df
# # print(len(mega_activity_df))
# # mega_activity_df = mega_activity_df.dropna()

# # print(len(mega_activity_df[mega_activity_df["ID"] <= 3]))
# activity_01 = get_activity_file(csv_files[0])
# print(activity_01)

In [ ]:
# Pre-processing: Activity

# features = extract_features(activity_01, column_id="ID", column_value="ACTIVITY", column_sort="TIMESTAMP", n_jobs=0, show_warnings=False)
# print(features.head())
# os.chdir(main_dir)
# features.to_csv("adhd_activity_features.csv", index=False, sep=",")    

# # Pts with motor activity recordings
# active_controls = controls_info[controls_info["ACC"] == 1]
# # print(active_controls)

# # Each pt has their own csv file...
# # demographics_01 = active_controls[active_controls["ID"] == 1]
# y = pd.Series(active_controls["ADHD"], index=active_controls["ID"])
# y = y.fillna(0)
# y["ADHD"] = active_controls["ADHD"]
# y.index = active_controls["ID"]
# y["ID"] = y.index
# print(y)
# print(list(y.index))
# print(demographics_01)
# print(activity_data_01)
# print(y.loc[1:3])

# activity_data_01.plot(figsize=(10,10))
# plt.show()

# Find relevant features using tsfresh
# relevant_features = extract_relevant_features(mega_activity_df[mega_activity_df["ID"] <= 3], y.loc[1:3], column_id='ID', column_sort='TIMESTAMP')

# relevant_features.head()

# X_full_train, X_full_test, y_train, y_test = train_test_split(relevant_features, y, test_size=.3, random_state=42)
# X_filtered_train = select_features(X_full_train, y_train)
# X_filtered_train.head()

In [ ]:
# # Splitting into training set and testing set

# # Don't let the machine know which ones are ADHD and which ones aren't
# X = controls_info.drop("ADHD", axis=1)
# y = controls_info["ADHD"]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify = y, random_state=2025)
